In [75]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Device configuration
device = torch.device('mps' if torch.mps.is_available() else 'cpu')
print(device)

mps


In [76]:
BATCH_SIZE = 100

# MNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.13, 0.31)])

train_dataset = datasets.MNIST(root='data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transform, download=True)

#train_dataset = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
#test_dataset = datasets.FashionMNIST(root='data', train=False, transform=transform, download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [77]:
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, 1024)
        self.fc4 = nn.Linear(1024, g_output_dim)
        
        self.apply(init_weights)
    
    # forward method
    def forward(self, x): 
        # x = F.leaky_relu(self.fc1(x), 0.2)
        # x = F.leaky_relu(self.fc2(x), 0.2)
        # x = F.leaky_relu(self.fc3(x), 0.2)
        # return torch.tanh(self.fc4(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return torch.tanh(self.fc4(x))
    
class Discriminator(nn.Module):
    def __init__(self, d_input_dim):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(d_input_dim, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features//2)
        self.fc4 = nn.Linear(self.fc3.out_features, 1)
        
        self.apply(init_weights)
    
    # forward method
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        return torch.sigmoid(self.fc4(x))

In [78]:
# build network
Z_DIM = 50
mnist_dim = train_dataset.train_data.size(1) * train_dataset.train_data.size(2)

G = Generator(g_input_dim = Z_DIM, g_output_dim = mnist_dim).to(device)
D = Discriminator(mnist_dim).to(device)

/opt/anaconda3/envs/datascience_env/lib/python3.12/site-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [79]:
G

Generator(
  (fc1): Linear(in_features=50, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)

In [80]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [81]:
# loss
criterion = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [82]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, mnist_dim), torch.ones(BATCH_SIZE, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on fake
    z = Variable(torch.randn(BATCH_SIZE, Z_DIM).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(BATCH_SIZE, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [83]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = Variable(torch.randn(BATCH_SIZE, Z_DIM).to(device))
    y = Variable(torch.ones(BATCH_SIZE, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

In [85]:
test_z = Variable(torch.randn(BATCH_SIZE, Z_DIM).to(device))


In [86]:
generated = G(test_z)

In [87]:
generated.shape

torch.Size([100, 784])

In [88]:
save_image(generated.view(generated.size(0), 1, 28, 28), 
                   f'./output/all.png')

In [89]:
def generate_test_image(epoch):
    with torch.no_grad():
        generated = G(test_z)
        save_image(generated.view(generated.size(0), 1, 28, 28)[0], 
                   f'./output/sample_{epoch}.png')

In [90]:
n_epoch = 300
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))
    
    generate_test_image(epoch)

[1/300]: loss_d: 0.633, loss_g: 3.900
[2/300]: loss_d: 0.120, loss_g: 10.316
[3/300]: loss_d: 0.129, loss_g: 8.270
[4/300]: loss_d: 0.087, loss_g: 10.855
[5/300]: loss_d: 0.081, loss_g: 11.509
[6/300]: loss_d: 0.075, loss_g: 10.853
[7/300]: loss_d: 0.059, loss_g: 12.008
[8/300]: loss_d: 0.057, loss_g: 11.081
[9/300]: loss_d: 0.042, loss_g: 11.590
[10/300]: loss_d: 0.052, loss_g: 11.768
[11/300]: loss_d: 0.044, loss_g: 10.850
[12/300]: loss_d: 0.046, loss_g: 11.118
[13/300]: loss_d: 0.044, loss_g: 11.528
[14/300]: loss_d: 0.048, loss_g: 10.335
[15/300]: loss_d: 0.049, loss_g: 9.877
[16/300]: loss_d: 0.038, loss_g: 10.535
[17/300]: loss_d: 0.039, loss_g: 10.059
[18/300]: loss_d: 0.039, loss_g: 9.574
[19/300]: loss_d: 0.039, loss_g: 10.207
[20/300]: loss_d: 0.042, loss_g: 10.052
[21/300]: loss_d: 0.045, loss_g: 10.033
[22/300]: loss_d: 0.041, loss_g: 9.799
[23/300]: loss_d: 0.048, loss_g: 9.125
[24/300]: loss_d: 0.051, loss_g: 8.957
[25/300]: loss_d: 0.053, loss_g: 9.428
[26/300]: loss_d: